# [1.1] Get the list of movies

In [57]:
from bs4 import BeautifulSoup
import requests
import json

In [58]:
soup = BeautifulSoup(open('C:/Users/leona/Desktop/ADMHMK-3/movies2.html'), "html.parser")
soup.head()
lst_a = soup.select('a')
urls = []
for i in lst_a:
    urls.append(i.get('href'))
urls[0]

'https://en.wikipedia.org/wiki/10_to_Midnight'

In [59]:
soup = BeautifulSoup(open('C:/Users/leona/Desktop/ADMHMK-3/movies1.html'), "html.parser")
soup.head()
lst_a = soup.select('a')
lst_a
for i in lst_a:
    urls.append(i.get('href'))
urls[10000]   

'https://en.wikipedia.org/wiki/Love_by_the_Light_of_the_Moon'

In [60]:
soup = BeautifulSoup(open('C:/Users/leona/Desktop/ADMHMK-3/movies3.html'), "html.parser")
soup.head()
lst_a = soup.select('a')
lst_a
for i in lst_a:
    urls.append(i.get('href'))
urls[20000] 

'https://en.wikipedia.org/wiki/Z.P.G.'

In [61]:
urls[29999]

'https://en.wikipedia.org/wiki/Whistle_(2003_film)'

In [70]:
dicturls = {}
for i in range(len(urls)):
    dicturls[i] = urls[i]
with open('dicturls.json', 'w') as fp:
    json.dump(dicturls, fp)
with open(r"C:\Users\leona\Desktop\ADMHMK-3\dicturls.json", 'r') as file:
    data = file.read()
dicturls = json.loads(data) 

In [72]:
dicturls[str(29999)]

'https://en.wikipedia.org/wiki/Whistle_(2003_film)'

# [1.2] Crawl Wikipedia

In [ ]:
from urllib.error import URLError, HTTPError, ContentTooShortError
import time
def getwikipageshtml(urls):
    k=0
    for i in range(len(urls)):
        try:
            ur_l = requests.get(urls[i])
            soup = BeautifulSoup(ur_l.content, 'html.parser')
            soup = soup.prettify("utf-8")   
            stringa = 'Articles/article-'+str(k)+'.html'
            k = k+1
            Html_file= open(stringa, "wb")
            Html_file.write(soup)
            Html_file.close()
        except(URLError,HTTPError, ContentTooShortError)  as e:
            html = None
        #time.sleep(0.001) #Actually for this task we don't need to stop anytime
    return
getwikipageshtml(urls)

In [7]:
len(urls)

30000

# [1.3] Parse downloaded pages

In [35]:
myfile = open("Articles/article-0.html")
#soup = BeautifulSoup(myfile, 'html.parser')

In [8]:
import csv
import io
for i in range(30000):
    filename = "Articles/article-"+str(i)+".html"
    with open(filename, encoding="utf-8") as f:
        data = f.read()
        soup = BeautifulSoup(data, 'html.parser')
        #Now That I opened the file I have to look for the section asked
        #first I take the title and clear him of spaces and the word -Wikipedia
        titlepage = soup.title.string
        titleonly = titlepage.split("- Wikipedia")
        titlepage = titleonly[0].strip()
        #now I create empty string as intro and plot
        intro = ''
        plot = ''
        #Now i searcvh the first paragraph that usually or is empty or is the intro
        start = soup.find('p')
        intro = start
        intro1 = start.text
        B = ''
        #in B I put m,y limit for the paragraphs in the intro, because after this h2 there will always be the plot
        B = intro.find_next_sibling('h2')
        if(B!=None and B.find_next_sibling('p')):
            C = B.find_next_sibling('p')
            while(C != intro.find_next_sibling('p')): 
                intro1 = intro1 + intro.find_next_sibling('p').text
                intro = intro.find_next_sibling('p')
            plot = ''    
            #then i do the same with the plot, so I start at B and end in the next h2
            plot = B
            plot1 = ''
            compare = ''
            if(B.find_next_sibling('h2')):
                compare = B.find_next_sibling('h2')
                compareto = compare.find_next_sibling('p')
                while(compareto != plot.find_next_sibling('p')):
                    plot1 = plot1 + plot.find_next_sibling('p').text
                    #print(plot1)
                    plot = plot.find_next_sibling('p')
                    #if plot or intro are empty I put NA
        if(intro1 == ''):
            intro1 = "NA"
        if plot1 == '':
            plot1 = "NA"
        #Now I start working on the other features
        intro = intro1
        plot = plot1
        film_name = 'NA'
        director = "NA"
        producer = "NA"
        writer = "NA"
        starring = "NA"
        music = "NA"
        release_date = "NA"
        runtime = "NA"
        country = "NA"
        language = "NA"
        budget = "NA"
#'film_name', 'director', 'producer','writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget'
        for link in soup.find_all('tr'):
            if soup.find('th',{'class': ['summary']})!= None:
                    film_name = soup.find('th',{'class': ['summary']} ).text.strip()
            if link.th:
#I just check in the th and if I find the class I need I take the relative td. Some of them are inaccessible so the if link.td
                if(link.th.text.strip() == 'Directed by'):
                     director = link.td.text.strip()
                elif(link.th.text.strip() == 'Produced by'):
                      producer = link.td.text.strip()
                elif(link.th.text.strip() == 'Written by'):
                    writer = link.td.text.strip()
                elif(link.th.text.strip() == 'Starring'):
                    starring = link.td.text.strip()
                elif(link.th.text.strip() == 'Music by'):
                     music = link.td.text.strip()               
                elif(link.th.text.strip() == 'Release date'):
                    if link.td:
                        release_date = link.td.text.strip()
                elif(link.th.text.strip() == 'Running time'):
                    runtime = link.td.text.strip()
                elif(link.th.text.strip() == 'Country'):
                    if link.td:
                        country = link.td.text.strip()
                elif(link.th.text.strip() == 'Language'):
                    if link.td:
                        language = link.td.text.strip()
                elif(link.th.text.strip() == 'Budget'):
                    budget = link.td.text.strip()
#now I open the tsv files and create one for every film.        
        tsvname = 'Tsvfiles/'+'film'+str(i)+'.tsv'
        with io.open(tsvname, "w", encoding="utf-8") as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(['title','intro', 'plot', 'film_name', 'director', 'producer','writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget'])
            tsv_writer.writerow([titlepage, intro, plot, film_name, director, producer, writer, starring, music, release_date, runtime, 
                 country, language, budget])

In [9]:
titlepage

'Whistle (2003 film)'

In [17]:
intro

'\n\n\n\n         What Became of Jack and Jill?\n        \n\n       is a 1972 British\n       \n        horror film\n       \n       directed by\n       \n        Bill Bain\n       \n       and starring\n       \n        Mona Washbourne\n       \n       ,\n       \n        Paul Nicholas\n       \n       , and\n       \n        Vanessa Howard\n       \n       .\n       \n\n         [1]\n        \n\n       It was part of an abandoned attempt by\n       \n        Amicus Pictures\n       \n       to compete with\n       \n        Hammer Studios\n       \n       by breaking into the\n       \n        grindhouse\n       \n       market. Studio executives were ultimately too disturbed by the final product to release it under the Amicus name, and they sold the film to\n       \n        20th Century Fox\n       \n       .\n      '

this is just to try stuff.

In [13]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [52]:
import pandas as pd
dataset1 = pd.read_csv('Cleantsv/filmclean-8.tsv', delimiter='\t')
#dataset1 = pd.read_csv('Tsvfiles/film1', delimiter='\t')
dataset1

,title,intro,plot,film_name,director,producer,writer,starring,music,release date,runtime,country,language,budget
0,big chill film,big chill 1983 american comedi drama film dire...,harold cooper bath young son wife dr sarah coo...,big chill,lawrenc kasdan,michael shamberg,lawrenc kasdan barbara benedek,tom bereng glenn close jeff goldblum william h...,bill conti,septemb 28 1983 1983 09 28,105 minut,unit state,english,8 million 1


In [35]:
words[13]

'budget 10 midnight'

I clean The files

In [10]:
import io 
import nltk
import csv
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
from nltk.stem import PorterStemmer 

 
ps = PorterStemmer() 
#the fuction preprocess the string as asked in the hmk
def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [ps.stem(w) for w in tokens if not w in stopwords.words('english')]
    return " ".join(filtered_words)
#I use the function on every section of the tsv files, except for the tiles of the categories, so from the part 13
for i in range(30000):
    file1 = open('Tsvfiles/film'+str(i)+'.tsv', encoding="utf8") 
    line = file1.read()# Use this to read file content as a stream: 
    words = line.split('\t') 
    for j in range(13, len(words)):
        words[j] = preprocess(words[j])
        if j ==13:
            #Here for the format of tsv files and my split('\t') the word budget would always be in my title, so I take her out
            words[j] = words[j].replace('budget ','')    
    tsvname = "Cleantsv/filmclean-"+str(i)+'.tsv'
    with io.open(tsvname, "w", encoding="utf-8") as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(['title','intro', 'plot', 'film_name', 'director', 'producer','writer', 'starring', 'music', 'release date', 'runtime', 'country', 'language', 'budget'])
            tsv_writer.writerow(words[13:])

In [12]:
words[13]

'whistl 2003 film'

# NOw I create the dictionary

In [13]:
dictionar = {}
k = 0
for i in range(30000):
    file1 = open("Cleantsv/filmclean-"+str(i)+'.tsv', encoding="utf8") 
    line = file1.read()# Use this to read file content as a stream: 
    #print(line)
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    #print(wordssplitted1, wordssplitted2)
    for i in wordssplitted1:
        #print(type(i))
        if i not in dictionar:
            dictionar[i] = str(k)
            k = k+1
    for i in wordssplitted2:
        #print(type(i))
        if i not in dictionar:
            dictionar[i] = str(k)
            k = k+1
#dictionar
import json

with open('Dictionary.json', 'w') as fp:
    json.dump(dictionar, fp)

In [9]:
import pandas as pd
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary.json", 'r') as file:
    data = file.read()
diction = json.loads(data)
#dictions = pd.DataFrame(diction)
diction['1913']

'12741'

In [15]:
len(diction)
type(diction['hom'])

str

# Here is the inverted Dictionary

In [16]:
dictionar2 = {}
length = 0
for i in range(30000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    file1 = open(file, encoding="utf8") 
    line = file1.read()# Use this to read file content as a stream: 
    #print(line)
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    #print(wordssplitted1, wordssplitted2)
    for j in wordssplitted1:
        code = diction[j]
        if code not in dictionar2:
            dictionar2[code] = [file]
        elif file not in dictionar2[code]:
            dictionar2[code].append(file)
    for j in wordssplitted2:
        code = diction[j]
        if code not in dictionar2:
            dictionar2[code] = [file]
        elif file not in dictionar2[code]:
            dictionar2[code].append(file)
#dictionar
import json

with open('Dictionary1.json', 'w') as fp:
    json.dump(dictionar2, fp)

In [10]:
import pandas as pd
import json
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary1.json", 'r') as file:
    data = file.read()
diction2 = json.loads(data)
#dictions = pd.DataFrame(diction)
#diction2['0']


In [120]:
diction['2019']
len(diction2)

114796

In [77]:
#diction2['7507']

# First engine searchengine1

In [74]:
with open(r"C:\Users\leona\Desktop\ADMHMK-3\dicturls.json", 'r') as file:
    data = file.read()
dicturls = json.loads(data) 
import io 
import nltk
import csv
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
from nltk.stem import PorterStemmer 

 
ps = PorterStemmer() 
#the fuction preprocess the string as asked in the hmk
def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [ps.stem(w) for w in tokens if not w in stopwords.words('english')]
    return " ".join(filtered_words)

In [76]:
#HERE i DEFINE THE FUCTION TO RECEIVE THE DATASET, GIVEN AN INPUT
y = list(input().split())
def searchengine1(y):
    for i in range(len(y)):
        y[i]= preprocess(str(y[i]))
    #Now I tranform the list of input in a list of the codes in the dictiionary based on the input
    yfinal=[] #use this because some words have no match in the vocabulary
    for i in range(len(y)):
        #print(y[i])
        if y[i] in diction:
            yfinal.append(diction[y[i]])
    #Now I have to search inside the lists of values from the keys i foundb and see if some films match in the various keys.
    if  len(yfinal)<len(y):
        return print('We are sorry there are no films, in my database, that match ALL the words you gave me !(')
    else:
        #print(yfinal)
        starting_values = diction2[yfinal[0]]
#print(starting_values)
        final_values = starting_values.copy()
        for codes in range(1,len(yfinal)):
        #print(codes)
            new = []
            for film in final_values:
            #print(film)
                if film in diction2[yfinal[codes]]:
                    new.append(film)
            final_values = new
        megaDataframe = pd.DataFrame(columns = ['Title', 'Intro', 'Url'])
    #megaDataframe
        if not final_values:
            return print("Wow no film matched my quiery, I need more films to compare!")
        else:
            k=0
            for document in final_values:
                totakeurl = document.replace('Cleantsv/filmclean-','')
                totakeurl = str(int(totakeurl.replace('.tsv', '')))
                url = dicturls[totakeurl]
                temporary = pd.read_csv('Tsvfiles/'+'film'+(totakeurl)+'.tsv',delimiter='\t' )
                title = temporary['title'][0]
                intro =  temporary['intro'][0].replace('\r\n','')
                new_row = [title, intro, url]
                megaDataframe.loc[k]=new_row
                k=k+1
            return megaDataframe
A = searchengine1(y)
A


Ross


,Title,Intro,Url
0,Superman III,Superman III is a Briti...,https://en.wikipedia.org/wiki/Superman_III
1,Footloose (1984 film),Footloose is a 1984 Ame...,https://en.wikipedia.org/wiki/Footloose_(1984_...
2,Protocol (film),Protocol is a 1984 Amer...,https://en.wikipedia.org/wiki/Protocol_(film)
3,Splash (film),Splash is a 1984 Americ...,https://en.wikipedia.org/wiki/Splash_(film)
4,Friday the 13th: A New Beginning,Friday the 13th: A New Beginning ...,https://en.wikipedia.org/wiki/Friday_the_13th:...
...,...,...,...
351,Turning Paige,Turning Paige is a 2001...,https://en.wikipedia.org/wiki/Turning_Paige
352,The Saddest Music in the World,The Saddest Music in the World ...,https://en.wikipedia.org/wiki/The_Saddest_Musi...
353,Goon (film),Goon is a 2011 Canadian...,https://en.wikipedia.org/wiki/Goon_(film)
354,Charming (film),Charming is a 2018 Cana...,https://en.wikipedia.org/wiki/Charming_(film)


# Building of the second search engine

In [2]:
import io 
import nltk
import csv
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 
from nltk.stem import PorterStemmer 

 
ps = PorterStemmer() 
#the fuction preprocess the string as asked in the hmk
def preprocess(sentence):
    sentence = sentence.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(sentence)
    filtered_words = [ps.stem(w) for w in tokens if not w in stopwords.words('english')]
    return " ".join(filtered_words)

In [15]:
Docum_and_words = []
for i in range(30000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    file1 = open(file, encoding="utf8") 
    line = file1.read()# Use this to read file content as a stream: 
    #print(line)
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    words = wordssplitted1+wordssplitted2
    A = " ".join(words)
    Docum_and_words.append(A)
#print(Docum_and_words) 
#I have the lis of documents with intro and plot



In [23]:
import math
from textblob import TextBlob as tb
def tf(word, doc):
    return doc.count(word) / len(doc)

def n_containing(word, doclist):
    return sum(1 for doc in doclist if word in doc.split())


# Create dictionary with idf to save computational costs

# Third dictionary

Some preliminary declarations to run the code at any moment.

In [81]:
import pandas as pd
import json
Docum_and_words = []
for i in range(30000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    file1 = open(file, encoding="utf8") 
    line = file1.read()
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    words = wordssplitted1+wordssplitted2
    A = " ".join(words)
    Docum_and_words.append(A)

In [82]:
Docum_and_words[0]

'10 midnight 1983 american crime horror thriller film 3 direct j lee thompson screenplay origin written william robert film star charl bronson lead role support cast includ lisa eilbach andrew steven gene davi geoffrey lewi wilford brimley 10 midnight releas citi film subsidiari cannon film american cinema march 11 1983 warren staci gene davi young offic equip repairman kill women reject sexual advanc attempt flirt alway seen creepi women result frequent reject 4 first victim betti june gilbert offic worker acquaint track wood area observ sex boyfriend ambush coupl kill boyfriend give chase nake woman catch stab death 4 two lo angel polic detect leo kessler charl bronson paul mcann andrew steven investig murder kessler season veteran forc mcann consider younger 4 staci avoid prosecut construct sound alibi assault victim nake except pair latex glove hide fingerprint thu minim evid lauri kessler lisa eilbach daughter leo acquaint victim student nurs becom target killer 4 mcann refus go a

In [80]:
with open(r"C:\Users\leona\Desktop\ADMHMK-3\dicturls.json", 'r') as file:
    data = file.read()
dicturls = json.loads(data) 
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary.json", 'r') as file:
    data = file.read()
diction = json.loads(data) #first dict with every word and a unique value
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary1.json", 'r') as file:
    data = file.read()
diction2 = json.loads(data)#first inverted dict with number and list of document with a word that has that number in diction
import math
from textblob import TextBlob as tb
def tf(word, doc):
    return doc.count(word) / len(doc)
def n_containing(word, doclist):
    return sum(1 for doc in doclist if word in doc.split())

def idf(word, doclist):
    return math.log(len(doclist) / float(n_containing(word, doclist)))

def tfidf(word, doc, doclist):
    return (tf(word, doc) * idf(word, doclist))

In [55]:
#diction

In [34]:
ncontain = {}
for i in diction:
    refer = diction[i]
    #print(i)
    ncontain[i] = len(diction2[refer])
with open('ncontain.json', 'w') as fp:
    json.dump(ncontain, fp)
with open(r"C:\Users\leona\Desktop\ADMHMK-3\ncontain.json", 'r') as file:
    data = file.read()
ncontain = json.loads(data) #first dict with every word and a unique value

In [38]:
len(ncontain.keys())

114796

In [44]:
import math
idfdict = {}
for i in ncontain:
    idfdict[i]=math.log(30000 / ncontain[i])
with open('idfdict.json', 'w') as fp:
    json.dump(idfdict, fp)
with open(r"C:\Users\leona\Desktop\ADMHMK-3\idfdict.json", 'r') as file:
    data = file.read()
idfdict = json.loads(data)

# Second dictionary with the tf-idf values

In [51]:
dictionar3 = {}
#length = 0
for i in range(30000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    temp = []
    for j in Docum_and_words[i].split():
            if j not in temp:
                code = diction[j]
                value = tf(j, Docum_and_words[i].split())*idfdict[j]
                li = [file, value]
                if code not in dictionar3:
                    dictionar3[code] = [file, value]
                else:                
                    dictionar3[code].append([file, value])
                temp.append(j)
import json

with open('Dictionary2.json', 'w') as fp:
    json.dump(dictionar3, fp)

In [52]:
len(dictionar3)

114796

# Now I import the files I need and prepare the code for the search engine2

In [ ]:
import pandas as pd
import json
with open(r"C:\Users\leona\Desktop\ADMHMK-3\dicturls.json", 'r') as file:
    data = file.read()
dicturls = json.loads(data) 
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary.json", 'r') as file:
    data = file.read()
diction = json.loads(data) #first dict with every word and a unique value
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary1.json", 'r') as file:
    data = file.read()
diction2 = json.loads(data)#first inverted dict with number and list of document with a word that has that number in diction
with open(r"C:\Users\leona\Desktop\ADMHMK-3\ncontain.json", 'r') as file:
    data = file.read()
ncontain = json.loads(data) #dict with number of times a word appear in all the cod
with open(r"C:\Users\leona\Desktop\ADMHMK-3\idfdict.json", 'r') as file:
    data = file.read()
idfdict = json.loads(data) #dict with the idf for every word
with open(r"C:\Users\leona\Desktop\ADMHMK-3\Dictionary2.json", 'r') as file:
    data = file.read()
diction3 = json.loads(data) #second inverted dictionary wit doc and tfidf for eevry word
import math

ASbout copsine similarity I have to take the tfidf of my k best document and multipèly singularly it with the values of the quiery that are tf(relative of the quiery)*idf(relative to my documents for each component)


The query entered by the user can also be represented as a vector. We will calculate the TF*IDF for the query

TF	IDF	TF*IDF
life	0.5	1.405507153	0.702753576
learning	0.5	1.405507153	0.702753576
Let us now calculate the cosine similarity of the query and Document1. You can do the calculation using this tool.

Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

Dot product(Query, Document1) 
     = ((0.702753576) * (0.140550715) + (0.702753576)*(0.140550715))
     = 0.197545035151

||Query|| = sqrt((0.702753576)2 + (0.702753576)2) = 0.993843638185

||Document1|| = sqrt((0.140550715)2 + (0.140550715)2) = 0.198768727354

Cosine Similarity(Query, Document) = 0.197545035151 / (0.993843638185) * (0.198768727354)
                                        = 0.197545035151 / 0.197545035151
                                        = 1
        
https://janav.wordpress.com/2013/10/27/tf-idf-and-cosine-similarity/

I prepare the code for the search engine

Here there are the functions to easily compute the cosine similarity given two vectors.

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as pyplot
from scipy.spatial.distance import cosine
def simple_dot(a, b):
    dsum = 0.
    for ((idx,), val) in np.ndenumerate(a):
        dsum += float(val) * float(b[idx])
    return dsum
def l2_norm(a):
    return math.sqrt(np.dot(a, a))
def cosine_similarity(a, b):
    return np.dot(a,b) / (l2_norm(a) * l2_norm(b))

I remind that Docume_and_words is a list with inside every intro-plot(preprocessed) for every document, so I can just accees to it for my research.

In [ ]:
import pandas as pd
import json
Docum_and_words = []
for i in range(30000):
    file = "Cleantsv/filmclean-"+str(i)+'.tsv'
    file1 = open(file, encoding="utf8") 
    line = file1.read()
    words = line.split('\t') 
    wordssplitted1 = words[14].split()
    wordssplitted2 = words[15].split()
    words = wordssplitted1+wordssplitted2
    A = " ".join(words)
    Docum_and_words.append(A)

Now I build the search engine2.

In [ ]:
import numpy
from sklearn.feature_extraction.text import TfidfVectorizer

y = list(input().split())
def searchengine2(y):
    for i in range(len(y)):
        y[i]= preprocess(str(y[i]))
    #Now I tranform the list of input in a list of the codes in the dictionary based on the input
    #NOw I need the tf for the cosine similarity(the idf is already ina dict)
    
    yfinal=[] #use this because some words have no match in the vocabulary
    for i in range(len(y)):
        #print(y[i])
        if y[i] in diction:
            yfinal.append(diction[y[i]])
    #Now I have to search inside the lists of values from the keys i foundb and see if some films match in the various keys.
    if  len(yfinal)<len(y):
        return print('We are sorry there are no films, in my database, that match ALL the words you gave me !(')
    else:
        starting_values = diction2[yfinal[0]]
        final_values = starting_values.copy()
        for codes in range(1,len(yfinal)):
            for film in final_values:
                if film not in diction2[yfinal[codes]]:
                    final_values.remove(film)
        megaDataframe = pd.DataFrame(columns = ['Title', 'Intro', 'Url','Score'])
        if not final_values:
            return print("Wow no film matched my quiery, I need more films to compare!")
        else:
            k=0
            for document in final_values:
                totakeurl = document.replace('Cleantsv/filmclean-','')
                totakeurl = int(totakeurl.replace('.tsv', ''))
                url = urls[totakeurl]
                temporary = pd.read_csv('Tsvfiles/'+'film'+str(totakeurl)+'.tsv',delimiter='\t' )
                with open(document, encoding="utf8") as file:
                    line = file.read()
                words = line.split('\t') 
                wordssplitted11 = words[14].split()
                wordssplitted21 = words[15].split()
                s2=str(wordssplitted11)+str(wordssplitted21)
                tfidf = TfidfVectorizer()
                s1 = diction.keys()
                response = tfidf.fit_transform([s2, str(s1)])
                feature_names = tfidf.get_feature_names()
               # for col in response.nonzero()[1]:
                    #print (feature_names[col], ' - ', response[0, col])
                title = temporary['title'][0]
                intro =  temporary['intro'][0].replace('\r\n','')
                new_row = [title, intro, url, feature_names]
                megaDataframe.loc[k]=new_row
                k=k+1
            return (megaDataframe)
searchengine2(y)